# incremental_enrollment_trial-processing_results
---------

결과 분석

## Environment

In [1]:
%load_ext autoreload
%autoreload 2
%pylab
%matplotlib inline

import pandas as pd
import pickle
import numpy as np
import sys
import os

Using matplotlib backend: TkAgg
Populating the interactive namespace from numpy and matplotlib


In [2]:
sys.path.append('/host/projects/sv_experiments/sv_system')
os.environ["CUDA_DEVICE_ORDER"]="PCI_BUS_ID"   # see issue #152
os.environ["CUDA_VISIBLE_DEVICES"]="1"

In [5]:
from utils import key2df
from utils import compute_eer

## Results

Meta data는 result.pkl에 저장되고, 

posScore(target sample에 대한 score), negScore(non-target sample에 대한 score), answer(verification output)만을 사용해서 결과를 뽑는다.

In [1]:
records = {}
base_dir = "icassp19_result/enr306_uttr1/base/""
for t_dir in os.listdir(base_dir):
    posScore = pickle.load(open(base_dir+t_dir+'/posScores.pkl', 'rb'))
    negScore = pickle.load(open(base_dir+t_dir+'/negScores.pkl', 'rb'))
    answers = pickle.load(open(base_dir+t_dir+'/answers.pkl', 'rb'))
    acc = answers[0]/(answers[0]+answers[1])
    eer = compute_eer(posScore, negScore)
    print("[base_{}] eer:{:.4f}, acc:{:.4f}".format(t_dir, eer*100, acc*100))
    records['base_'+t_dir] = (eer, acc)

SyntaxError: EOL while scanning string literal (<ipython-input-1-f306a330c53c>, line 2)

In [29]:
inc_records = {}
base_dir = "adaptive_sv_system/results/sortedPos/"
for enr in ['3','5','7', '9']: # n_enr cases
    subpath = 'n_enr'+enr+'_results/'
    for n in sorted(os.listdir(base_dir+subpath+'inc/')):
        for t_dir in sorted(os.listdir(base_dir+subpath+'inc/'+ n)):
            posScore = pickle.load(open(base_dir+subpath+'inc/'+n+'/'+t_dir+'/posScores.pkl', 'rb'))
            negScore = pickle.load(open(base_dir+subpath+'inc/'+n+'/'+t_dir+'/negScores.pkl', 'rb'))
            answers = pickle.load(open(base_dir+subpath+'inc/'+n+'/'+t_dir+'/answers.pkl', 'rb'))
            acc = answers[0]/(answers[0]+answers[1])
            eer = compute_eer(posScore, negScore)
            print('inc_{}: eer: {:.5f}, acc: {:.5f}'.format(t_dir, eer*100, acc*100))
            inc_records[n+'_inc_'+t_dir] = (eer, acc)

inc_n_enr_3_pRatio_0.01: eer: 23.25179, acc: 91.40107
inc_n_enr_3_pRatio_0.01_initEnr: eer: 5.97688, acc: 93.50839
inc_n_enr_3_pRatio_0.01_initEnr_thresUpdt: eer: 5.89847, acc: 94.48203
inc_n_enr_3_pRatio_0.01_thresUpdt: eer: 23.65971, acc: 92.59116
inc_n_enr_3_pRatio_0.1: eer: 6.38161, acc: 93.52068
inc_n_enr_3_pRatio_0.1_initEnr: eer: 4.20992, acc: 94.17560
inc_n_enr_3_pRatio_0.1_initEnr_thresUpdt: eer: 4.24624, acc: 95.17737
inc_n_enr_3_pRatio_0.1_thresUpdt: eer: 6.55011, acc: 94.46570
inc_n_enr_3_pRatio_0.5: eer: 4.79223, acc: 94.80880
inc_n_enr_3_pRatio_0.5_initEnr: eer: 4.09836, acc: 95.35176
inc_n_enr_3_pRatio_0.5_initEnr_thresUpdt: eer: 4.10773, acc: 95.78460
inc_n_enr_3_pRatio_0.5_thresUpdt: eer: 4.78272, acc: 95.22680
inc_n_enr_3_pRatio_0.9: eer: 4.72146, acc: 95.73246
inc_n_enr_3_pRatio_0.9_initEnr: eer: 4.03991, acc: 96.48286
inc_n_enr_3_pRatio_0.9_initEnr_thresUpdt: eer: 4.05973, acc: 96.32106
inc_n_enr_3_pRatio_0.9_thresUpdt: eer: 4.73212, acc: 95.44460
inc_n_enr_5_pRatio

### Write results to txt

In [27]:
eer_string = ''
acc_string = ''
#for trial in ['1','2','3','4','5']:
for trial in ['1']:
    eer_string += 'Q3\tQ5\tQ7\tQ9\tQ3\tQ5\tQ7\tQ9\tQ3\tQ5\tQ7\tQ9\tQ3\tQ5\tQ7\tQ9\n'
    acc_string += 'Q3\tQ5\tQ7\tQ9\tQ3\tQ5\tQ7\tQ9\tQ3\tQ5\tQ7\tQ9\tQ3\tQ5\tQ7\tQ9\n'
    for option in ['', '_initEnr', '_thresUpdt', '_initEnr_thresUpdt']:
        for ratio in ['0.01', '0.1', '0.5', '0.9']:
            for enr in ['3', '5', '7', '9']:
                key = trial + '_inc_n_enr_' + enr + '_pRatio_' + ratio + option
                #if key in inc_records.keys():
                eer = inc_records[key][0]
                acc = inc_records[key][1]
                eer_string += '{:.5f}'.format(eer*100) + '\t'
                acc_string += '{:.5f}'.format(acc*100) + '\t'

        eer_string += '\n'
        acc_string += '\n'
    eer_string += '\n'
    acc_string += '\n'

with open(base_dir + 'acc_excel.txt', 'w') as f:
    f.write(acc_string)
with open(base_dir + 'eer_excel.txt', 'w') as f:
    f.write(eer_string)